In [2]:
pip install schedule

  Obtaining dependency information for schedule from https://files.pythonhosted.org/packages/20/a7/84c96b61fd13205f2cafbe263cdb2745965974bdf3e0078f121dfeca5f02/schedule-1.2.2-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import time
import schedule
import sqlite3
from datetime import datetime
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker

DATABASE_URL = "sqlite:///./temp.db"  # Update this URL to match your database
BACKUP_DIR = 'backups'

def get_engine():
    return create_engine(DATABASE_URL)

def backup_table(engine, table_name):
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    backup_file = os.path.join(BACKUP_DIR, f"{table_name}_backup_{timestamp}.sql")

    with sqlite3.connect(engine.url.database) as conn:
        with open(backup_file, 'w') as f:
            for line in conn.iterdump():
                if line.startswith(f'INSERT INTO "{table_name}"'):
                    f.write(f'{line}\n')

    print(f"Backup for table {table_name} created at {backup_file}")

def backup_database():
    if not os.path.exists(BACKUP_DIR):
        os.makedirs(BACKUP_DIR)

    engine = get_engine()
    metadata = MetaData(bind=engine)
    metadata.reflect()

    for table_name in metadata.tables.keys():
        backup_table(engine, table_name)

schedule.every(5).minutes.do(backup_database)

def run_scheduler():
    while True:
        schedule.run_pending()
        time.sleep(1)

def restore_table(engine, table_name, backup_file):
    with sqlite3.connect(engine.url.database) as conn:
        cursor = conn.cursor()
        
        # Remove existing data from the table
        cursor.execute(f'DELETE FROM "{table_name}";')
        
        # Restore data from the backup file
        with open(backup_file, 'r') as f:
            sql_script = f.read()
            conn.executescript(sql_script)

    print(f"Data for table {table_name} restored from {backup_file}")
